# Rototranslation

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from icepy4d.least_squares import rototra3d

from icepy4d.utils.transformations import Rotrotranslation

icepy_path = Path.cwd().parents[1]

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Initialize the Rototranslation class

Create the Rototranslation object by passing a 3D transformation matrix to constrution, as: 

```
rotra = Rotrotranslation(Tmat)
```

The 3D transformation matrix is a x4 numpy array expressed in homogeneous coordinates (last row is [0, 0, 0, 1]) and must contain both the rotion matrix and the traslation vector, as:

```
Tmat =  r11 | r12 | r13 | tx
        r21 | r22 | r23 | ty
        r31 | r32 | r33 | tz
        0   | 0   | 0   | 1
```

For the Belvedere Glacier, the transformation between the carthesian local reference system and the WGS84 UTM32N reference system are stored within the Rototranslation class. 
Therefore, the Rototranslation object can be directly initialized as:

```
rotra_loc2UTM = Rotrotranslation.belvedere_loc2utm()
rotra_UTM2loc = Rotrotranslation.belvedere_utm2loc()

```

The transformation matrix can be accessed as:

```
rotra.T


In [2]:
belv_rotra = Rotrotranslation.belvedere_loc2utm()
print(belv_rotra.T)

[[ 7.06579328e-01 -7.06873715e-01 -1.26001140e-04  4.16614833e+05]
 [ 7.06873715e-01  7.06579268e-01  2.02054813e-04  5.09093271e+06]
 [-5.38263700e-05 -2.31959390e-04  9.99462247e-01  1.76754700e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


The Rototranslation object can be also initialized by reading a transformation matrix from a file, as:

```
rotra = Rotrotranslation.read_T_from_file(filename)
```

The file must be a simple text file containing the transformation matrix, structured as a 4x4 matrix with whitespace-separated values.
To keep the method simple, no other file formats are currently supported.
This structure of the transformation matrix is the same as that you get by copying and pasting to an empty text file the transformation matrix from CloudCompare 'Apply Transformation' tool, for instance:

```
0.706579327583 -0.70687371492 -0.00012600114 416614.833
0.706873714924 0.706579267979 0.000202054813 5090932.706
-5.382637e-05 -0.00023195939 0.999462246895 1767.547
0.0 0.0 0.0 1.0
```


In [11]:
rotra_from_file = Rotrotranslation.read_T_from_file(
    icepy_path / "data/crs/BELV_LOC2UTM.txt"
)
print(rotra_from_file.T)

[[ 7.06579328e-01 -7.06873715e-01 -1.26001140e-04  4.16614833e+05]
 [ 7.06873715e-01  7.06579268e-01  2.02054813e-04  5.09093271e+06]
 [-5.38263700e-05 -2.31959390e-04  9.99462247e-01  1.76754700e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


## Working with points as numpy arrays

After having initialized the rototranslation object, the transformation can be applied to a point or a list of points.
To this end, let's first read the series of point from a text file by using pandas and extract the point coordinates as a numpy array with shape (N, 3), where N is the number of points.
The points can then be easily transformed with the `transform` method of the Rototranslation class.

In [3]:
fname = "data/targets/target_world.csv"

target_loc= pd.read_csv(icepy_path / fname)
points = target_loc[["X", "Y", "Z"]].to_numpy()
print(f"Points in starting reference system:\n{points}")

points_utm = belv_rotra.transform(points)
print(f"Points in destination reference system:\n{points_utm}")

Points in starting reference system:
[[-499.855   402.0301  240.3745]
 [-302.8139  442.8938  221.9927]
 [-543.471   365.4121  217.7905]
 [  49.6488  192.0875   71.7466]
 [ -15.5549  315.3171  117.3462]
 [  41.5596  274.7651   92.0252]
 [  53.2401  163.0449   78.296 ]
 [-194.7426  287.7045  150.1151]
 [ -47.1266  257.7534  119.0749]
 [ -21.9544  227.5956   95.207 ]
 [ -19.5291  238.1006   96.6108]
 [   8.2768  244.675    80.6632]
 [  -6.8296  285.013    98.518 ]
 [-141.9356  324.2512  141.5215]
 [  -1.7781  219.4006   74.5789]
 [  60.112   211.5654   71.8052]
 [-532.7409  391.02    238.8015]
 [  51.1682  210.4649   70.9032]
 [  -1.8212  219.2598   74.5584]]
Points in destination reference system:
[[4.15977431e+05 5.09086349e+06 2.00772589e+03]
 [4.16087773e+05 5.09103164e+06 1.98933389e+03]
 [4.15972500e+05 5.09080678e+06 1.98516487e+03]
 [4.16514123e+05 5.09110354e+06 1.83920779e+03]
 [4.16380938e+05 5.09114453e+06 1.88475779e+03]
 [4.16449962e+05 5.09115625e+06 1.85945674e+03]
 [4.165

## Working with point clouds

The RotoTranslation class can also be used to transform point clouds. To this end, the point cloud is read by using the Open3D library and then transformed with the `transform_pcd` method of the Rototranslation class.
The Rototranslation class already implements functions to read and write the point cloud with Open3D, therefore the transformation is straightforward as:

```
pcd_transformed = rotra.transform_pcd(pcd, out_path)
```

If the output path is not specified, the point cloud is not saved to disk.

In [6]:
# Define paths
pcd_path = icepy_path / "res/monthly_pcd/belvedere2021_densaMedium_lingua_50cm_utm.ply"
out_path = icepy_path /"res/monthly_pcd/belvedere2021_densaMedium_lingua_50cm_loc.ply"

# Apply the transformation
pcd_transf = belv_rotra.transform_pcd(pcd_path, out_path=out_path)

# Visualize the results
import open3d as o3d

o3d.visualization.draw_geometries([pcd_transf])